# Constraint Satisfaction Problems

In [1]:
from copy import copy, deepcopy
from itertools import combinations

In [3]:
VarsA = ["A", "B", "C", "D", "E"]
DomainsA = {v: [i for i in range(10)] for v in VarsA}
ConstraintsA = [(list(p), lambda x,y: x != y) for p in combinations(VarsA, 2)] 
ConstraintsA.append((["A","B"], lambda a, b: a + b == 10))
ConstraintsA.append((["B","D"], lambda b, d: b + d == 6))
ConstraintsA.append((["C"], lambda c: c < 5))
ConstraintsA.append((["A"], lambda a: a > 5))
ConstraintsA.append((["A","B","C","D","E"], lambda a, b, c, d, e: a + b + c + d + e == 30))
MathProblem = {"Vars": VarsA, "Domains": DomainsA, "Constraints": ConstraintsA}

In [4]:
VarsC = ["France", "Germany", "Loux", "Belgium", "Netherlands"]
DomainsC = {v: ["blue", "red", "yellow", "green"] for v in VarsC}
ConstraintsC = []
for (a, b) in [("France", "Germany"), ("France", "Belgium"), ("France", "Loux"),
               ("Belgium", "Netherlands"), ("Belgium", "Loux"), ("Belgium", "Germany"),
               ("Loux", "Germany"), ("Netherlands", "Germany")]:
    ConstraintsC.append(([a, b], lambda a, b: a != b))
ColoringProblem = {"Vars": VarsC, "Domains": DomainsC, "Constraints": ConstraintsC}

### Task 0


In [6]:
def get_constraints(var, constraints):
    lista_constraints = []
    for i in constraints:
        country, f = i
        for c in country:
            if (c == var):
                lista_constraints.append(i)
    return lista_constraints

get_constraints("France", ConstraintsC)

[(['France', 'Germany'], <function __main__.<lambda>(a, b)>),
 (['France', 'Belgium'], <function __main__.<lambda>(a, b)>),
 (['France', 'Loux'], <function __main__.<lambda>(a, b)>)]

### Task 1


In [7]:
def fixed_constraints(solution, constraints):
    lista = []
    for i in solution.keys():
        lista.append(i)
        
    lista_completa=[]
    for country, f in constraints:
        for value in country:
            lista_completa.append((country, f))
        
    sterse = []    
    for i in lista_completa:
        country,f = i
        for l in country:
            if l not in lista:
                sterse.append(i)
                
    returned_list = []
    for const in constraints:
        if const not in sterse:
            returned_list.append(const)
    return returned_list

print(fixed_constraints({"France": "blue", "Belgium": "green"}, ConstraintsC)) 
print(fixed_constraints({"A": "1", "C": "2"}, ConstraintsA)) 

[(['France', 'Belgium'], <function <lambda> at 0x000001CECDCA1940>)]
[(['A', 'C'], <function <listcomp>.<lambda> at 0x000001CECDCA1CA0>), (['C'], <function <lambda> at 0x000001CECDCA0280>), (['A'], <function <lambda> at 0x000001CECDCA0310>)]


### Task 2

In [8]:
def check_constraint(solution, constraint):
    country, f = constraint
    lista = []
    for c in country:
        lista.append(solution[c])
    return f(*lista)

print(check_constraint({"France": "blue", "Belgium": "green"}, ConstraintsC[1])) 
print(check_constraint({"France": "blue", "Belgium": "blue"}, ConstraintsC[1]))
print(check_constraint({"C": 10, "A": 10}, ConstraintsA[-2]))
print(check_constraint({"C": 10, "A": 3}, ConstraintsA[-2])) 

True
False
True
False


### Task 3: PCSP algorithm

In [10]:
def PCSP(vars, domains, constraints, acceptable_cost, solution, cost):
    global best_solution
    global best_cost
    if not vars:
        print("New best: " + str(cost) + " - " + str(solution))
        
        best_solution = solution
        best_cost = cost
        
        if best_cost < acceptable_cost:
            return True
    
    elif not domains[vars[0]]:
        return False
    elif cost == best_cost:
        return False
    else:
        var = vars[0]
        val = domains[var].pop(0)

        new_solution = deepcopy(solution)
        new_solution[var] = val

        fixed_ones = fixed_constraints(new_solution, get_constraints(var, constraints))
        
        new_cost = cost
        for constraint in fixed_ones:
            if check_constraint(new_solution, constraint)==False:
                new_cost += 1
                
        if best_cost>new_cost:
            if PCSP(deepcopy(vars[1:len(vars)]), deepcopy(domains), constraints, acceptable_cost, new_solution, new_cost):
                return True
        return PCSP(vars, domains, constraints, acceptable_cost, solution, cost)


    
def run_pcsp(problem, acceptable_cost):
    global best_solution
    global best_cost

    [vars, domains, constraints] = [problem[e] for e in ["Vars", "Domains", "Constraints"]]
    
    best_solution = {}
    best_cost = len(constraints)

    if PCSP(vars, deepcopy(domains), constraints, acceptable_cost, {}, 0):
        print("Best found: " + str(best_cost) + " - " + str(best_solution))
    else:
        print("Acceptable solution not found; " + "Best found: " + str(best_cost) + " - " + str(best_solution))
        
run_pcsp(MathProblem, 1)
run_pcsp(ColoringProblem, 1)

New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Acceptable solution not found; Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
New best: 6 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red'}
New best: 4 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue'}
New best: 3 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow'}
New best: 2 - {'France': 'blue'

Expected output for numbers:

```
New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
```

Expected output for country colors:

```
New best:  8  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  6  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
New best:  4  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  3  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  2  -  {'Loux': 'red', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
Best found: 1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
```